In [1]:
# Package imports
import numpy as np
import copy
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
import pandas as pd
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [129]:
data = pd.read_csv("data_vectors_10000.csv")
print("data type:",type(data))
print("data shape:", data.shape)
print("data:",data)

# Split the data into train and test
train_dataset = data.sample(frac=0.8, random_state=0)
test_dataset = data.drop(train_dataset.index)

# Normalize (divide by max value)
x_train = train_dataset.multiply(1/100).iloc[:,0:6]
y_train = train_dataset.multiply(1/100).iloc[:,6:12]
x_test = test_dataset.multiply(1/100).iloc[:,0:6]
y_test = test_dataset.multiply(1/100).iloc[:,6:12]
print('Train shape',x_train.shape)
print('Test shape',x_test.shape)

data type: <class 'pandas.core.frame.DataFrame'>
data shape: (10000, 12)
data:       x1  x2  x3  x4  x5  x6  y1  y2  y3  y4  y5  y6
0      0   1   0   0   4   7   2   1   3   3   4   7
1      1   0   2   2   0   0   1   1   2   2   3   5
2      0   0   3   3   5   0   1   2   3   3   5   8
3      2   1   0   0   6   0   2   1   5   3   6   9
4      4   1   2   0   0   0   4   1   2   5   3   8
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
9995   0   0  23  44   0  90  21  23  23  44  46  90
9996   0  24   0   0  48  83  11  24  24  35  48  83
9997   0  23   0   0  43  90  24  23  20  47  43  90
9998   0   0  24  44   0  90  22  22  24  44  46  90
9999   0   0  24   0  48  96  24  24  24  48  48  96

[10000 rows x 12 columns]
(10000, 12) (8000, 12) (2000, 12)
(8000, 6)


In [ ]:
# Define Sequential Model
model = keras.Sequential(
    [   keras.Input(shape=(6,)),
        keras.layers.Dense(48, activation='relu'),
        keras.layers.Dense(96, activation='relu'),
        keras.layers.Dense(192, activation='relu'),
        keras.layers.Dense(96, activation='relu'),
        keras.layers.Dense(48, activation='relu'),
        keras.layers.Dense(6)
    ]
)
model.summary()

In [131]:
# loss, optimizer and metrics
model.compile(loss=keras.losses.MeanSquaredError(), 
              optimizer=keras.optimizers.Adam(learning_rate=0.001), 
              metrics=['mae'])

In [132]:
class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') <= 9.0e-08):
            print("\n\n\nReached 9.0e-08 loss value, stopping training!\n\n")
            self.model.stop_training = True
trainingStopCallback = haltCallback()

In [ ]:
# training
model.fit(x_train, y_train, epochs=10000,batch_size=64, verbose=1, callbacks=[trainingStopCallback])

In [134]:
pred = model.predict(x_train, verbose=0)
test_result = np.round(pred*100).astype(int) - np.round(y_train*100).astype(int)
test_result_row_sums = test_result.abs().sum(axis=1)
num_correct=len(test_result_row_sums[test_result_row_sums == 0])
print('Performance (train set):', (100*num_correct)/test_result.shape[0], '%')

pred = model.predict(x_test, verbose=0)
y_round = np.round(y_test*100).astype(int)
test_result = np.round(pred*100).astype(int) - np.round(y_test*100).astype(int)
test_result_row_sums = test_result.abs().sum(axis=1)
num_correct= len(test_result_row_sums[test_result_row_sums == 0])
print('Performance (test set):', (100*num_correct)/test_result.shape[0], '%')


Performance (train set): 100.0 %
Performance (test set): 99.2 %


In [137]:
# Check prediction of first test input
x1 = x_test.iloc[0:1,:]
y1 = y_test.iloc[0:1,:]
x1_pred = model.predict(x1, verbose=0)
print('x1',(x1*100).astype(int).to_string(index=False))
print('y1',(y1*100).astype(int).to_string(index=False))
print('prediction: \n',np.round(x1_pred*100).astype(int)[0,:])



x1  x1  x2  x3  x4  x5  x6
  0   1   0   0   4   7
y1  y1  y2  y3  y4  y5  y6
  2   1   3   3   4   7
prediction: 
 [2 1 3 3 4 7]


In [ ]:
model.save("mathpyramid_vectors_99.2_correct")

In [ ]:
#model = keras.models.load_model("mathpyramid_1987_correct")
#model.summary()

In [136]:
# Predict with higher values (<= 1000)
data = {'x1': [175],
        'x2': [114],
        'x3': [102],
        'x4': [175+114],
        'x5': [114 + 102],
        'x6': [175+114 + 114 + 102]
        }  
# Create the pandas DataFrame
df = pd.DataFrame(data)
print(df)
pred = model.predict(df/1000)
print(np.round(pred*1000).astype(int))

    x1   x2   x3   x4   x5   x6
0  175  114  102  289  216  505
1/1 [==============================] - 0s 16ms/step
[[157 155  94 311 249 560]]
